In [124]:
physLessonsByDate = {}  # словарь {scheduled_time: [(id, event_id), (id, event_id), ...]}
avrgVoteForLesson = {}  # словарь {lesson_id: average tech_quality}


with open("lessons.txt") as records:
    for record in records:
        # рассматриваем только уроки по физике
        if "phys" in record:
            record = [field.strip() for field in record.split("|")]
            date = record[3].split(maxsplit=1)[0]
            
            if date not in physLessonsByDate:
                physLessonsByDate[date] = []

            physLessonsByDate[date].append((record[0], record[1]))


with open("quality.txt") as records:
    for record in records.readlines()[2:-2]:
        lessonId, mark = tuple([field.strip() for field in record.split("|")])
        try:
            mark = float(mark)
        except ValueError:
            continue
        
        avrgVoteForLesson[lessonId] \
            = (avrgVoteForLesson.setdefault(lessonId, mark) + mark) / 2


# извлекаем из "users.txt" только id репетиторов
tutorsIds = set()
with open("users.txt") as records:
    for record in records.readlines()[2:-2]:
        if "tutor" in record:
            tutorId = record.split("|")[0].strip()
            tutorsIds.add(tutorId)


# создаем словарь поиска id репетитора по event_id
tutorsByEventId = {} # словарь {event_id: user_id}
with open("participants.txt") as records:
    for record in records.readlines()[2:-2]:
        eventId, userId = tuple([field.strip() for field in record.split("|")])
        if userId in tutorsIds:
            tutorsByEventId[eventId] = userId
                                

result = []

tutorDict = {} # словарь  {tutor : [avrg vote, avrg vote, ...]} 
for date, lessons in physLessonsByDate.items():
    for lessonInfo in lessons:
        lessonId, eventId = lessonInfo
        
        tutor = tutorsByEventId[eventId]
        if tutor not in tutorDict:
            tutorDict[tutor] = []
        
        try:
            tutorDict[tutor].append(avrgVoteForLesson[lessonId])
        except KeyError:
            continue
     
    for tutor, votes in tutorDict.items():
        if len(votes) != 0:
            tutorDict[tutor] = sum(votes) / len(votes)
        else:
            tutorDict[tutor] = float("inf")
    
        
    badTutor = min(tutorDict.keys(), key=(lambda k: tutorDict[k]))
    
    result.append([date, badTutor, tutorDict[badTutor]])
    
    tutorDict.clear()
    
result = sorted(result, key=(lambda l: l[0]))

print(*result, sep="\n")

['2020-01-11', '8fe03f08-8581-430c-a590-9888ab36deb3', 4.5]
['2020-01-12', '696c838e-c054-4e9f-a51a-50bf5660f364', 4.9]
['2020-01-13', '696c838e-c054-4e9f-a51a-50bf5660f364', 5.0]
['2020-01-14', 'c6718d0e-976c-4d6c-b0e0-32c770776567', 4.0]
['2020-01-15', 'b37ccae8-fc31-4ad8-8f55-ca855b23fbf6', 5.0]
['2020-01-16', '2fa2ab62-f1b0-4036-872f-bcfd9a8686ff', 4.0]
['2020-01-17', '696c838e-c054-4e9f-a51a-50bf5660f364', 4.5]
['2020-01-18', '43efce48-94b2-4412-857f-223d45969008', 4.0]
['2020-01-19', 'be676776-8366-4c71-8a35-d58014806eb5', 4.5]
['2020-01-20', '30a19496-bdaf-461c-abbc-2709ae520201', 4.5]
